# The analysis below seeks to identify the most popular hour of the day for picking up a shared electric car (Bluecar) in the city of Paris over the month of April 2018.

In [2]:
# Import the pandas library
import pandas as pd

# Import the numpy library
import numpy as np

In [4]:
# Import the dataset file
df=pd.read_csv('/content/Autolib_dataset.csv')

In [ ]:
# Preview a sample of the dataset
df.sample(5)

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
1548,35 rue Mathis,0,0,0,0,0,nonexistent,Paris,NaN,paris-mathis-35,STATION,"48.8911334, 2.3725951",75019,Paris/Mathis/35,operational,NaN,2,station,ok,nonexistent,2018,4,1,13,10
1982,13 Rue de Lens,4,4,0,0,0,nonexistent,Nanterre,NaN,nanterre-lens-13,STATION,"48.8998826, 2.2242026",92000,Nanterre/Lens/13,operational,NaN,2,station,ok,nonexistent,2018,4,9,2,57
2374,169 avenue Gabriel PÃ©ri,0,0,0,0,0,nonexistent,Gennevilliers,NaN,gennevilliers-gabrielperi-169,SPACE,"48.9242205, 2.2942668",92230,Gennevilliers/Gabriel PÃ©ri/169,broken,NaN,0,full_station,closed,broken,2018,4,5,22,14
4691,1 avenue Victor Hugo,5,5,0,0,0,nonexistent,Noisy-le-Sec,NaN,noisylesec-victorhugo-1,STATION,"48.8920896, 2.4674469",93130,Noisy-le-Sec/Victor Hugo/1,operational,NaN,0,station,ok,nonexistent,2018,4,8,19,16
193,24 boulevard Diderot,0,0,0,0,1,operational,Paris,NaN,paris-diderot-24,SPACE,"48.8458586, 2.3748769",75012,Paris/Diderot/24,operational,NaN,3,full_station,ok,operational,2018,4,9,20,28


In [5]:
# Preview the dataset information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Address              5000 non-null   object
 1   Cars                 5000 non-null   int64 
 2   Bluecar counter      5000 non-null   int64 
 3   Utilib counter       5000 non-null   int64 
 4   Utilib 1.4 counter   5000 non-null   int64 
 5   Charge Slots         5000 non-null   int64 
 6   Charging Status      5000 non-null   object
 7   City                 5000 non-null   object
 8   Displayed comment    111 non-null    object
 9   ID                   5000 non-null   object
 10  Kind                 5000 non-null   object
 11  Geo point            5000 non-null   object
 12  Postal code          5000 non-null   int64 
 13  Public name          5000 non-null   object
 14  Rental status        5000 non-null   object
 15  Scheduled at         47 non-null     object
 16  Slots 

In [6]:
# Dropping irrelevant data
df.drop(['Cars', 'Charge Slots', 'Charging Status', 'Displayed comment', 'ID', 'Geo point', 
         'Public name', 'Scheduled at', 'Slots', 'Station type', 'Subscription status'], axis=1, inplace=True)
df.head(5)

,Address,Bluecar counter,Utilib counter,Utilib 1.4 counter,City,Kind,Postal code,Rental status,Status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,Paris,STATION,75015,operational,ok,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,0,0,Paris,STATION,75014,operational,ok,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,0,2,Le Bourget,STATION,93350,operational,ok,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,1,0,Paris,STATION,75013,operational,ok,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,0,0,Paris,PARKING,75017,operational,ok,2018,4,8,17,23


In [7]:
# The Status column shows the no of resources available for "scheduled", which is the status if there is a "scheduled at" date.
# We check whether there are any resources available for closed stations. The analysis below shows that there are no vehicles 
# associated with a closed station.
Status = df[df['Status'] == 'closed']
print(Status.head(5))
print(Status.all())
print(df.shape)

                          Address  Bluecar counter  ...  hour  minute
6                  37 rue Leblanc                0  ...    18      20
29               35 rue de Boucry                0  ...     2      29
46  81 avenue de la Grande ArmÃ©e                0  ...    21       2
68                  6 avenue Foch                0  ...     3      11
80                    4 Rue Lobau                0  ...    14       3

[5 rows x 14 columns]
Address                True
Bluecar counter       False
Utilib counter        False
Utilib 1.4 counter    False
City                   True
Kind                   True
Postal code            True
Rental status          True
Status                 True
year                   True
month                  True
day                    True
hour                  False
minute                False
dtype: bool
(5000, 14)


In [8]:
# We thus go ahead and delete all the rows where Status is showing "closed"
df_filtered = df[df['Status'] != 'closed']
print(df_filtered)

                                 Address  Bluecar counter  ...  hour  minute
0                    2 Avenue de Suffren                0  ...    11      43
1              145 Rue Raymond Losserand                6  ...     7      24
2       2 Avenue John Fitzgerald Kennedy                3  ...    20      14
3                 51 Rue EugÃ¨ne OudinÃ©                3  ...     4      37
4     6 avenue de la Porte de Champerret                3  ...    17      23
...                                  ...              ...  ...   ...     ...
4995                 8 avenue MÃ©nelotte                2  ...    11      26
4996                   37 rue de Dantzig                4  ...    16      56
4997                      142 rue du Bac                1  ...     7       1
4998         2 avenue du Val de Fontenay                2  ...    17      27
4999                 5 Rue Henry Brisson                4  ...    10       2

[4670 rows x 14 columns]


In [9]:
# The analysis is checking the most popular time to rent a car. That means that there must be a car at the station to rent in the first place
# Check the dataset to see whether there are rows where the "Bluecar counter" and the "Utilib counter" and the "Utilib 1.4 counter" are all zero


Status2 = df_filtered[df_filtered['Bluecar counter'] + df_filtered['Utilib counter'] + df_filtered['Utilib 1.4 counter'] == 0]
Status2

,Address,Bluecar counter,Utilib counter,Utilib 1.4 counter,City,Kind,Postal code,Rental status,Status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,Paris,STATION,75015,operational,ok,2018,4,8,11,43
5,8 Boulevard Voltaire,0,0,0,Paris,STATION,75011,operational,ok,2018,4,6,7,2
10,14 rue Censier,0,0,0,Paris,STATION,75005,operational,ok,2018,4,1,4,39
13,34 avenue Jean Moulin,0,0,0,Paris,STATION,75014,operational,ok,2018,4,8,21,54
14,8 Boulevard Richard Lenoir,0,0,0,Paris,STATION,75011,operational,ok,2018,4,6,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4977,1 rue de Chanaleilles,0,0,0,Paris,STATION,75007,operational,ok,2018,4,2,19,51
4980,204 rue Saint-Martin,0,0,0,Paris,STATION,75003,operational,ok,2018,4,8,3,14
4990,47 boulevard de l'HÃ´pital,0,0,0,Paris,STATION,75013,operational,ok,2018,4,3,21,13
4992,3 avenue d'Ivry,0,0,0,Paris,STATION,75013,operational,ok,2018,4,9,23,1


In [10]:
# Drop the rows where the "Bluecar counter" and the "Utilib counter" and the "Utilib 1.4 counter" are all zero
df_filtered2 = df_filtered[df_filtered['Bluecar counter'] + df_filtered['Utilib counter'] + df_filtered['Utilib 1.4 counter'] != 0]
print(df_filtered2)

                                 Address  Bluecar counter  ...  hour  minute
1              145 Rue Raymond Losserand                6  ...     7      24
2       2 Avenue John Fitzgerald Kennedy                3  ...    20      14
3                 51 Rue EugÃ¨ne OudinÃ©                3  ...     4      37
4     6 avenue de la Porte de Champerret                3  ...    17      23
7            17 Rue des Luaps ProlongÃ©e                3  ...    22      13
...                                  ...              ...  ...   ...     ...
4995                 8 avenue MÃ©nelotte                2  ...    11      26
4996                   37 rue de Dantzig                4  ...    16      56
4997                      142 rue du Bac                1  ...     7       1
4998         2 avenue du Val de Fontenay                2  ...    17      27
4999                 5 Rue Henry Brisson                4  ...    10       2

[3651 rows x 14 columns]


In [11]:
# Check the dataset for null values
df_filtered2.isnull().sum()

Address               0
Bluecar counter       0
Utilib counter        0
Utilib 1.4 counter    0
City                  0
Kind                  0
Postal code           0
Rental status         0
Status                0
year                  0
month                 0
day                   0
hour                  0
minute                0
dtype: int64

In [12]:
# Concatenate the year, month, day, hour and minute into one column and give them new column a datetype format

df_filtered2['Date_Time'] = pd.to_datetime(df_filtered2[['year', 'month', 'day', 'hour', 'minute']])

df_filtered2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Address,Bluecar counter,Utilib counter,Utilib 1.4 counter,City,Kind,Postal code,Rental status,Status,year,month,day,hour,minute,Date_Time
1,145 Rue Raymond Losserand,6,0,0,Paris,STATION,75014,operational,ok,2018,4,6,7,24,2018-04-06 07:24:00
2,2 Avenue John Fitzgerald Kennedy,3,0,2,Le Bourget,STATION,93350,operational,ok,2018,4,3,20,14,2018-04-03 20:14:00
3,51 Rue EugÃ¨ne OudinÃ©,3,1,0,Paris,STATION,75013,operational,ok,2018,4,4,4,37,2018-04-04 04:37:00
4,6 avenue de la Porte de Champerret,3,0,0,Paris,PARKING,75017,operational,ok,2018,4,8,17,23,2018-04-08 17:23:00
7,17 Rue des Luaps ProlongÃ©e,3,1,0,Nanterre,STATION,92000,operational,ok,2018,4,4,22,13,2018-04-04 22:13:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,8 avenue MÃ©nelotte,2,0,0,Colombes,STATION,92700,operational,ok,2018,4,6,11,26,2018-04-06 11:26:00
4996,37 rue de Dantzig,4,0,0,Paris,STATION,75015,operational,ok,2018,4,4,16,56,2018-04-04 16:56:00
4997,142 rue du Bac,1,0,0,Paris,STATION,75007,operational,ok,2018,4,1,7,1,2018-04-01 07:01:00
4998,2 avenue du Val de Fontenay,2,0,0,Fontenay-Sous-Bois,STATION,94120,operational,ok,2018,4,4,17,27,2018-04-04 17:27:00


In [13]:
# Check the info
df_filtered2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3651 entries, 1 to 4999
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Address             3651 non-null   object        
 1   Bluecar counter     3651 non-null   int64         
 2   Utilib counter      3651 non-null   int64         
 3   Utilib 1.4 counter  3651 non-null   int64         
 4   City                3651 non-null   object        
 5   Kind                3651 non-null   object        
 6   Postal code         3651 non-null   int64         
 7   Rental status       3651 non-null   object        
 8   Status              3651 non-null   object        
 9   year                3651 non-null   int64         
 10  month               3651 non-null   int64         
 11  day                 3651 non-null   int64         
 12  hour                3651 non-null   int64         
 13  minute              3651 non-null   int64       

In [14]:
# Drop the original date and time columns as they are now irrelevant

df_filtered2.drop(['year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)

df_filtered2


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Address,Bluecar counter,Utilib counter,Utilib 1.4 counter,City,Kind,Postal code,Rental status,Status,Date_Time
1,145 Rue Raymond Losserand,6,0,0,Paris,STATION,75014,operational,ok,2018-04-06 07:24:00
2,2 Avenue John Fitzgerald Kennedy,3,0,2,Le Bourget,STATION,93350,operational,ok,2018-04-03 20:14:00
3,51 Rue EugÃ¨ne OudinÃ©,3,1,0,Paris,STATION,75013,operational,ok,2018-04-04 04:37:00
4,6 avenue de la Porte de Champerret,3,0,0,Paris,PARKING,75017,operational,ok,2018-04-08 17:23:00
7,17 Rue des Luaps ProlongÃ©e,3,1,0,Nanterre,STATION,92000,operational,ok,2018-04-04 22:13:00
...,...,...,...,...,...,...,...,...,...,...
4995,8 avenue MÃ©nelotte,2,0,0,Colombes,STATION,92700,operational,ok,2018-04-06 11:26:00
4996,37 rue de Dantzig,4,0,0,Paris,STATION,75015,operational,ok,2018-04-04 16:56:00
4997,142 rue du Bac,1,0,0,Paris,STATION,75007,operational,ok,2018-04-01 07:01:00
4998,2 avenue du Val de Fontenay,2,0,0,Fontenay-Sous-Bois,STATION,94120,operational,ok,2018-04-04 17:27:00


In [15]:
# Split the datetime into date and time

df_filtered2 ['Date'] = [d.date() for d in df_filtered2['Date_Time']]
df_filtered2 ['Time'] = [d.time() for d in df_filtered2['Date_Time']]

# Ensure that the new columns are in the required datetime format
df_filtered2['Date'] = pd.to_datetime(df_filtered2['Date'])
df_filtered2['Time'] = pd.to_datetime(df_filtered2['Time'], format='%H:%M:%S:%f', errors='ignore')


# Preview the data
df_filtered2.head(3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

,Address,Bluecar counter,Utilib counter,Utilib 1.4 counter,City,Kind,Postal code,Rental status,Status,Date_Time,Date,Time
1,145 Rue Raymond Losserand,6,0,0,Paris,STATION,75014,operational,ok,2018-04-06 07:24:00,2018-04-06,07:24:00
2,2 Avenue John Fitzgerald Kennedy,3,0,2,Le Bourget,STATION,93350,operational,ok,2018-04-03 20:14:00,2018-04-03,20:14:00
3,51 Rue EugÃ¨ne OudinÃ©,3,1,0,Paris,STATION,75013,operational,ok,2018-04-04 04:37:00,2018-04-04,04:37:00


In [16]:
# Check the info

df_filtered2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3651 entries, 1 to 4999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Address             3651 non-null   object        
 1   Bluecar counter     3651 non-null   int64         
 2   Utilib counter      3651 non-null   int64         
 3   Utilib 1.4 counter  3651 non-null   int64         
 4   City                3651 non-null   object        
 5   Kind                3651 non-null   object        
 6   Postal code         3651 non-null   int64         
 7   Rental status       3651 non-null   object        
 8   Status              3651 non-null   object        
 9   Date_Time           3651 non-null   datetime64[ns]
 10  Date                3651 non-null   datetime64[ns]
 11  Time                3651 non-null   object        
dtypes: datetime64[ns](2), int64(4), object(6)
memory usage: 370.8+ KB


In [17]:
# Strip the columns of leading and lagging spaces and convert the column headers into lower case
df_filtered2.columns = df_filtered2.columns.str.strip().str.lower()
df_filtered2.head(3)

,address,bluecar counter,utilib counter,utilib 1.4 counter,city,kind,postal code,rental status,status,date_time,date,time
1,145 Rue Raymond Losserand,6,0,0,Paris,STATION,75014,operational,ok,2018-04-06 07:24:00,2018-04-06,07:24:00
2,2 Avenue John Fitzgerald Kennedy,3,0,2,Le Bourget,STATION,93350,operational,ok,2018-04-03 20:14:00,2018-04-03,20:14:00
3,51 Rue EugÃ¨ne OudinÃ©,3,1,0,Paris,STATION,75013,operational,ok,2018-04-04 04:37:00,2018-04-04,04:37:00


In [18]:
# Exporting cleaned dataset to CSV
df_filtered2.to_csv('Bluecar.csv')
print(df_filtered2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3651 entries, 1 to 4999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   address             3651 non-null   object        
 1   bluecar counter     3651 non-null   int64         
 2   utilib counter      3651 non-null   int64         
 3   utilib 1.4 counter  3651 non-null   int64         
 4   city                3651 non-null   object        
 5   kind                3651 non-null   object        
 6   postal code         3651 non-null   int64         
 7   rental status       3651 non-null   object        
 8   status              3651 non-null   object        
 9   date_time           3651 non-null   datetime64[ns]
 10  date                3651 non-null   datetime64[ns]
 11  time                3651 non-null   object        
dtypes: datetime64[ns](2), int64(4), object(6)
memory usage: 370.8+ KB
None


In [28]:
# Re-importing the cleaned dataset to carry out the analysis and previewing the data

df_cleaned = pd.read_csv('Bluecar.csv')

df_cleaned.head(3)
print(df_cleaned.info())

# Ensure that the new columns are in the required datetime format
df_cleaned['date'] = pd.to_datetime(df_cleaned['date'])
df_cleaned['time'] = pd.to_datetime(df_cleaned['time'], format='%H:%M:%S:%f', errors='ignore')

print(df_cleaned.info())
df_cleaned.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3651 entries, 0 to 3650
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          3651 non-null   int64 
 1   address             3651 non-null   object
 2   bluecar counter     3651 non-null   int64 
 3   utilib counter      3651 non-null   int64 
 4   utilib 1.4 counter  3651 non-null   int64 
 5   city                3651 non-null   object
 6   kind                3651 non-null   object
 7   postal code         3651 non-null   int64 
 8   rental status       3651 non-null   object
 9   status              3651 non-null   object
 10  date_time           3651 non-null   object
 11  date                3651 non-null   object
 12  time                3651 non-null   object
dtypes: int64(5), object(8)
memory usage: 370.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3651 entries, 0 to 3650
Data columns (total 13 columns):
 #   Colu

(3651, 13)

In [23]:
# Drop column "Unnamed: 0" which seems to have been added automatically in the process

df_cleaned.drop(['Unnamed: 0'], inplace=True, axis =1)

In [29]:
df_cleaned.drop_duplicates(inplace=True)
df_cleaned.shape
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3651 entries, 0 to 3650
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Unnamed: 0          3651 non-null   int64         
 1   address             3651 non-null   object        
 2   bluecar counter     3651 non-null   int64         
 3   utilib counter      3651 non-null   int64         
 4   utilib 1.4 counter  3651 non-null   int64         
 5   city                3651 non-null   object        
 6   kind                3651 non-null   object        
 7   postal code         3651 non-null   int64         
 8   rental status       3651 non-null   object        
 9   status              3651 non-null   object        
 10  date_time           3651 non-null   object        
 11  date                3651 non-null   datetime64[ns]
 12  time                3651 non-null   object        
dtypes: datetime64[ns](1), int64(5), object(7)
memory

In [30]:
# From our dataset, select only the rows where the city = Paris, as this is the only dataset we are interested in
Paris = df_cleaned[df_cleaned['city'] == 'Paris']
Paris.sample(2)


,Unnamed: 0,address,bluecar counter,utilib counter,utilib 1.4 counter,city,kind,postal code,rental status,status,date_time,date,time
548,717,142 rue du Bac,5,0,0,Paris,STATION,75007,operational,ok,2018-04-05 12:05:00,2018-04-05,12:05:00
1076,1455,21 Quai de la Tournelle,1,0,0,Paris,STATION,75005,operational,ok,2018-04-06 06:34:00,2018-04-06,06:34:00


In [36]:
# Identify the most popular time that cars are dropped (for the Bluecar)

bluecar_popular_time = Paris.groupby(['time'])['bluecar counter'].count()

bluecar_popular_time.nlargest(10)

time
10:55:00    6
18:48:00    6
01:03:00    5
02:35:00    5
02:53:00    5
05:52:00    5
08:01:00    5
13:24:00    5
21:08:00    5
22:12:00    5
Name: bluecar counter, dtype: int64

In [38]:
# Identify the most popular time that cars are dropped (for the utilib)

utilib_popular_time = Paris.groupby(['time'])['utilib counter'].count()

utilib_popular_time.nlargest(10)

time
10:55:00    6
18:48:00    6
01:03:00    5
02:35:00    5
02:53:00    5
05:52:00    5
08:01:00    5
13:24:00    5
21:08:00    5
22:12:00    5
Name: utilib counter, dtype: int64

In [40]:
# Identify the most popular time that cars are dropped (for the utilib 1.4)

utilib2_popular_time = Paris.groupby(['time'])['utilib 1.4 counter'].count()

utilib2_popular_time.nlargest(10)

time
10:55:00    6
18:48:00    6
01:03:00    5
02:35:00    5
02:53:00    5
05:52:00    5
08:01:00    5
13:24:00    5
21:08:00    5
22:12:00    5
Name: utilib 1.4 counter, dtype: int64

In [37]:
# Identify the most popular station overall for the Bluecar

bluecar_popular_station = Paris.groupby(['bluecar counter','kind', 'address'])['address'].count()

bluecar_popular_station.nlargest(10)

bluecar counter  kind     address                   
4                STATION  1 rue des Bois                6
1                STATION  23 rue des Bernardins         4
                          35 rue Mathis                 4
                          8 Place du 25 AoÃ»t 1944      4
                          8 avenue de Choisy            4
2                SPACE    50 Boulevard Pereire          4
                 STATION  12 rue Chernoviz              4
                          15 avenue Ãmile Deschanel    4
                          3 avenue d'Ivry               4
                          46 quai de la RapÃ©e          4
Name: address, dtype: int64

In [39]:
# Identify the most popular station overall for the utilib

bluecar_popular_station = Paris.groupby(['utilib counter','kind', 'address'])['address'].count()

bluecar_popular_station.nlargest(10)

utilib counter  kind     address                    
0               STATION  19 rue BrÃ©montier             10
                         28 rue Darcet                  10
                         2 Avenue de Taillebourg         9
                         6 Rue Adolphe Yvon              9
                         1 rue des Bois                  8
                         12 rue Chernoviz                8
                         126 Avenue Philippe Auguste     8
                         182 Boulevard Voltaire          8
                         1bis rue Ravignan               8
                         229 Rue Raymond Losserand       8
Name: address, dtype: int64

In [41]:
# Identify the most popular station overall for the utilib 1.4

bluecar_popular_station = Paris.groupby(['utilib 1.4 counter','kind', 'address'])['address'].count()

bluecar_popular_station.nlargest(10)

utilib 1.4 counter  kind     address                     
0                   STATION  28 rue Darcet                   10
                             19 rue BrÃ©montier               9
                             2 Avenue de Taillebourg          9
                             6 Rue Adolphe Yvon               9
                             1 rue des Bois                   8
                             12 rue Chernoviz                 8
                             126 Avenue Philippe Auguste      8
                             182 Boulevard Voltaire           8
                             42 bis avenue Georges Mandel     8
                             68 avenue Mozart                 8
Name: address, dtype: int64